# Cài đặt RAG

## Cài đặt

In [1]:
!pip install langchain-qdrant
!pip install accelerate bitsandbytes
!pip install bert_score
!pip install xformers==0.0.28.post3 --index-url https://download.pytorch.org/whl/cu121
!pip install optimum qdrant-client wikipedia FastAPI uvicorn pyngrok
!pip install --upgrade pydantic
!pip install vllm
!pip install ragas
!pip install -U \
      python-dotenv \
      langchain \
      langchain_openai \
      langchain_community \
      langchain-huggingface \
      langchain-google-genai \
      streamlit \
      faiss-cpu \
      sentence-transformers \
      pypdf \
      docx2txt\
      vllm\
      triton
!pip install autoawq
!pip install accelerate transformers

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)


## Khởi tạo biến môi trường

In [ ]:
#Khai báo khóa API key
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"]  = ''

In [ ]:
# Cài đặt thông tin truy cập vector database
QDRANT_API_KEY = ""
QDRANT_URL = ""
EMBEDDINGS_MODEL_NAME="intfloat/multilingual-e5-base"
HUGGINGFACE_API_KEY= ""
QDRANT_COLLECTION_NAME = "ITUS_e5_600"
GENERATE_MODEL_NAME="gemini"
NGROK_STATIC_DOMAIN = ""
NGROK_TOKEN=""

## RAG gemini

In [4]:
from langchain.schema.document import Document
from langchain.schema.retriever import BaseRetriever
from langchain.retrievers import WikipediaRetriever
from langchain_qdrant import QdrantVectorStore
from langchain.llms import HuggingFacePipeline
from qdrant_client import QdrantClient
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.chains import RetrievalQA, MultiRetrievalQAChain
from langchain.llms import VLLM
from langchain.llms import HuggingFaceHub
from typing import List
import asyncio
from pydantic import BaseModel, Field
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from asyncio import to_thread
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain

class LLMServe:
    def __init__(self, model_name=GENERATE_MODEL_NAME) -> None:
        self.embeddings = self.load_embeddings()
        self.current_source = "qdrant"
        self.retriever = self.load_retriever(retriever_name=self.current_source, embeddings=self.embeddings)
        self.pipe = self.load_model_pipeline(model_name=model_name, max_new_tokens=384)
        self.prompt = self.load_prompt_template()
        self.rag_pipeline = self.load_rag_pipeline(
            llm=self.pipe,
            retriever=self.retriever,
            prompt=self.prompt
        )

    def load_embeddings(self):
        embeddings = HuggingFaceInferenceAPIEmbeddings(
            model_name=EMBEDDINGS_MODEL_NAME,
            api_key=HUGGINGFACE_API_KEY,
        )
        return embeddings

    def load_retriever(self, retriever_name, embeddings):
        """
        Load and create appropriate retriever based on retriever_name
        """
        base_retriever = None
        if retriever_name == "wiki":
            base_retriever = WikipediaRetriever(
                lang="vi",
                doc_content_chars_max=800,
                top_k_results=10
            )
        elif retriever_name == "qdrant":
            client = QdrantClient(
                url=QDRANT_URL,
                api_key=QDRANT_API_KEY,
                prefer_grpc=False
            )

            db = QdrantVectorStore(
                client=client,
                embedding=embeddings,
                collection_name=QDRANT_COLLECTION_NAME
            )
            base_retriever = db.as_retriever(search_kwargs={"k": 10})

        return base_retriever

    def load_model_pipeline(self, model_name="gemini", max_new_tokens=384):
        """
        Load and create appropriate model pipeline based on model_name
        """
        if model_name == "gemini":
            llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
        else:
            llm = VLLM(
                model=model_name,
                trust_remote_code=True,
                max_new_tokens=max_new_tokens,
                top_k=10,
                top_p=0.95,
                temperature=0.4,
                dtype="half",
            )
        return llm

    def load_prompt_template(self):
        query_template = """
                    <|system|>
                    '''Thực hiện trả lời câu hỏi từ thông tin có trong ngữ cảnh được cho. Chú ý các yêu cầu sau:
                    - Câu trả lời phải chính xác, đầy đủ và có liên quan đến câu hỏi.
                    - Nếu có nhiều ngữ cảnh chứa thông tin liên quan thì kết hợp các ngữ cảnh để tổng hợp.
                    - Chỉ sử dụng các thông tin có trong ngữ cảnh được cung cấp.
                    - Nếu ngữ cảnh không chứa thông tin về câu trả lời thì từ chối trả lời và không suy luận gì thêm và cung cấp thông tin liên lạc của khoa như sau để người dùng liên lạc:
                    "Vui lòng liên lạc Khoa Công Nghệ Thông Tin, trường Đại học Khoa Học Tự Nhiên - Đại học Quốc Gia TP.Hồ Chí Minh để giải đáp:
                    Địa chỉ: Phòng I.54, toà nhà I, 227 Nguyễn Văn Cừ, Q.5, TP.HCM
                    Điện thoại: (028) 62884499
                    Email: info@fit.hcmus.edu.vn"
                    </s>
                    <|user|>
                    Hãy trả lời câu hỏi sau dựa trên ngữ cảnh sau:

                    {context}
                    ---

                    Câu hỏi: {question}
                    </s>
                    <|assistant|>
                    """
        prompt = PromptTemplate(template=query_template, input_variables=["context", "question"])
        return prompt

    def load_rag_pipeline(self, llm, retriever, prompt):
        rag_pipeline = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=retriever,
            chain_type_kwargs={
                "prompt": prompt
            },
            return_source_documents=True,
        )
        return rag_pipeline

    def rag(self, source):
        if source == self.current_source:
            return self.rag_pipeline
        else:
            self.retriever = self.load_retriever(retriever_name=source, embeddings=self.embeddings)
            self.rag_pipeline = self.load_rag_pipeline(
                llm=self.pipe,
                retriever=self.retriever,
                prompt=self.prompt
            )
            self.current_source = source
            return self.rag_pipeline

In [5]:
!pip install triton

In [6]:
# Có thể thay đổi model khác bằng cách đổi tên model
#Ví dụ:
#model_name = GENERATE_MODEL_NAME
app = LLMServe(model_name="gemini")

# sever

In [7]:
from typing import Union, Optional
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder
from fastapi import FastAPI, HTTPException

# Định nghĩa các nguồn dữ liệu hợp lệ
VALID_SOURCES = ["qdrant", "wiki"]

origins = ["*"]
app_api = FastAPI()
app_api.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], # Cho phép tất cả các nguồn
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app_api.get("/")
def read_root():
    return {"message": "API RAG is running"}

@app_api.get("/rag/{source}")
async def read_item(source: str, q: Optional[str] = None):
    if source not in VALID_SOURCES:
        raise HTTPException(
            status_code=400,
            detail=f"Invalid source. Must be one of: {VALID_SOURCES}"
        )

    if not q:
        raise HTTPException(
            status_code=400,
            detail="Query parameter 'q' is required"
        )

    try:
        print(f"Debug - Processing request for source: {source}, query: {q}")
        rag_chain = app.rag(source=source)
        print(f"Debug - RAG chain type: {type(rag_chain)}")
        data = rag_chain.invoke({"query": q})

        sources = []
        for docs in data["source_documents"]:
            sources.append(docs.to_json()["kwargs"])

        return JSONResponse(content=jsonable_encoder({
            "result": data["result"],
            "source_documents": sources
        }))

    except Exception as e:
        print(f"Detailed error in endpoint: {str(e)}")
        print(f"Error type: {type(e)}")
        import traceback
        print(f"Traceback: {traceback.format_exc()}")
        raise HTTPException(
            status_code=500,
            detail=f"An error occurred: {str(e)}"
        )

In [10]:
!ngrok kill
!ngrok start --all

ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your account at https://dash

In [11]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.set_auth_token(NGROK_TOKEN)
ngrok_tunnel = ngrok.connect(8000,domain=NGROK_STATIC_DOMAIN)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app_api, port=8000)

INFO:     Started server process [5074]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://hugely-immense-bedbug.ngrok-free.app
INFO:     2402:800:63e4:18:71d2:ef6:ff76:e25c:0 - "OPTIONS /rag/qdrant?q=Sinh%20vi%C3%AAn%20c%E1%BA%A7n%20chu%E1%BA%A9n%20tr%C3%ACnh%20%C4%91%E1%BB%99%20ngo%E1%BA%A1i%20ng%E1%BB%AF%20l%C3%A0%20bao%20nhi%C3%AAu%20%C4%91%E1%BB%83%20t%E1%BB%91t%20nghi%E1%BB%87p? HTTP/1.1" 200 OK
Debug - Processing request for source: qdrant, query: Sinh viên cần chuẩn trình độ ngoại ngữ là bao nhiêu để tốt nghiệp?
Debug - RAG chain type: <class 'langchain.chains.retrieval_qa.base.RetrievalQA'>
INFO:     2402:800:63e4:18:71d2:ef6:ff76:e25c:0 - "GET /rag/qdrant?q=Sinh%20vi%C3%AAn%20c%E1%BA%A7n%20chu%E1%BA%A9n%20tr%C3%ACnh%20%C4%91%E1%BB%99%20ngo%E1%BA%A1i%20ng%E1%BB%AF%20l%C3%A0%20bao%20nhi%C3%AAu%20%C4%91%E1%BB%83%20t%E1%BB%91t%20nghi%E1%BB%87p? HTTP/1.1" 200 OK
INFO:     2402:800:63e4:18:71d2:ef6:ff76:e25c:0 - "OPTIONS /rag/qdrant?q=M%C3%B4n%20h%E1%BB%8Dc%20n%C3%A0o%20thu%E1%BB%99c%20H%E1%BB%8Dc%20k%E1%BB%B3%201%20n%C4%83m%204%20chuy%C3%AAn%20ng%C3%

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5074]
